In [ ]:
!pip install keras-tuner scikit-learn
!pip install icd10-cm bgen_reader pydantic
from google.colab import drive
drive.mount('/content/gdrive')
import tensorflow as tf
import kerastuner as kt
import sys
import os
import shutil
import string
import sklearn
sys.path.append(os.path.abspath('/content/gdrive/My Drive/6.874 project/Colab'))
import input_processing

     |████████████████████████████████| 71kB 3.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=7e80426034a6c62afe3e2fa58714d2266e6659f432fab8aa8b94da6c1bb6e356
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=6c38f413f7b48ad9888abeeda1d6db96f3694f34a74e5f1c8b2da3474a516460
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables
     |████████████████████████████████| 675kB 6.7MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 10.1MB 7.7MB/s 
     |████████████████████████████████| 768kB 48.7MB/s 
     |████████████████████████████████| 532kB 47.6MB/s 
     |████████████████████████████████| 286kB 41.7MB/s 
     |███████████████████

Mapping genotypes: 100%|██████████| 1261158/1261158 [01:14<00:00, 16927.43it/s]


reading -- time=0:00:01.49, thread 1 of 2, part 5 of 5


## Useful Constants

* A = number of different genotypic alleles
* P = number of different phenotypes
* D = dropout rate
* l2 = L2 regularization rate

In [ ]:
A = 1000
P = 1

## Network Architecture
Outputs:

1. Input: N x A x 3
1. Dense Layer w/ ReLU: N x A // 3
1. Dropout(D): N x A // 3
1. Dense Layer w/ ReLU: N x P * 3
1. Dense Layer w/ Sigmoid: N x P

In [ ]:
def make_CNN_model(c1_shape, c1_out, mp_size, mp_stride, dropout, l2):
    regularizer = tf.keras.regularizers.L2(l2)
    return tf.keras.Sequential(layers=[
        tf.keras.layers.InputLayer(input_shape=(A, 3)),
        tf.keras.layers.Conv1D(c1_out, c1_shape, activation=tf.nn.relu),
        tf.keras.layers.MaxPool1D(mp_size, mp_stride),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(P, activation=tf.nn.sigmoid, kernel_regularizer=regularizer)
    ], name='cnn_model')

## Model Training Parameters
#### Loss Function
Binary Cross-Entropy + L2 Regularization
#### Optimizer
Adam
#### Metric
Binary Accuracy

In [ ]:
def compile_model(model, optimizer):
    model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy', tf.metrics.AUC()])
    model.summary()

## Hyperparameter Tuning

In [ ]:
batch_size = 100
epochs = 20
use_multiprocessing=False
validation_split = 0.2

In [ ]:
class IntRange:
  def __init__(self, min, max, step):
    self.min = min
    self.max = max
    self.step = step
  
  def to_Int(self, hp, name):
    return hp.Int(name, min_value = self.min, max_value = self.max, step = self.step)

def get_tunable_CNN_model(hp, c1, kcount, mp_size, mp_stride):
    c1 = c1.to_Int(hp, 'c1_shape')
    mp_size = mp_size.to_Int(hp, 'max_pool_size')
    mp_stride = mp_stride.to_Int(hp, 'max_pool_stride')
    dropout = hp.Choice('dropout', values=[0.2, 0.5, 0.8])
    l2 = hp.Choice('l2', values=[0.01, 0.005, 0.001])
    lr = hp.Choice('lr', values=[0.01, 0.005, 0.001])
    model = make_CNN_model(c1, kcount, mp_size, mp_stride, dropout, l2)
    compile_model(model, tf.keras.optimizers.Adam(lr))
    return model

def get_tuner(model_builder):
    try:
      shutil.rmtree('hp_tuning')
    except FileNotFoundError:
      pass
    return kt.Hyperband(model_builder, objective=kt.Objective('val_auc', direction='max'), max_epochs=epochs, executions_per_trial=3, directory='hp_tuning', project_name='initial_model')

## Search Space

In [ ]:
c1_search = IntRange(3, P * 27, 2)
kcount = 32
mp_size = IntRange(2, A // 4, 2)
mp_step = IntRange(2, A // 4, 2)

## Get Training Data

In [ ]:
#def get_data(N, A, P):
 #   genotypes = tf.stack([tf.constant([[1, 0, 0]] * A, dtype=tf.float32)] * N)
  #  phenotypes = tf.stack([tf.constant([1] * P, dtype=tf.float32)] * N)
   # return genotypes, phenotypes

## Hyperparameter Tuning Call

In [ ]:
search_data = input_processing.get_data(A, 1000)
search_train_x, search_validation_x, search_train_y, search_validation_y = sklearn.model_selection.train_test_split(*search_data, test_size=0.2, random_state=0, stratify=search_data[1])

In [ ]:
tuner = get_tuner(lambda hp: get_tunable_CNN_model(hp, c1_search, kcount, mp_size, mp_step))
tuner.search_space_summary()
tuner.search(search_train_x, search_train_y, epochs=epochs, validation_data=(search_validation_x, search_validation_y))
tuner.results_summary()

Trial 30 Complete [00h 01m 01s]
val_auc: 0.6984615723292033

Best val_auc So Far: 0.7394871910413107
Total elapsed time: 00h 19m 41s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in hp_tuning/initial_model
Showing 10 best trials
Objective(name='val_auc', direction='max')
Trial summary
Hyperparameters:
c1_shape: 21
max_pool_size: 166
max_pool_stride: 158
dropout: 0.8
l2: 0.005
lr: 0.005
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 2aea7c78d7a4b884aa434ec86365da1b
Score: 0.7394871910413107
Trial summary
Hyperparameters:
c1_shape: 23
max_pool_size: 74
max_pool_stride: 60
dropout: 0.5
l2: 0.005
lr: 0.001
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 32a5e89a55b8300b16db15a58cff02ca
Score: 0.7317948937416077
Trial summary
Hyperparameters:
c1_shape: 23
max_pool_size: 78
max_pool_stride: 204
dropout: 0.5
l2: 0.01
lr: 0.01
tuner/epochs: 20
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0


## Optimal Model

In [ ]:
models = tuner.get_best_models()

Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 969, 21)           2037      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 6, 21)             0         
_________________________________________________________________
flatten (Flatten)            (None, 126)               0         
_________________________________________________________________
dropout (Dropout)            (None, 126)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 127       
Total params: 2,164
Trainable params: 2,164
Non-trainable params: 0
_________________________________________________________________


## Train Model

In [ ]:
x, y = input_processing.get_data(A, 100000)
train_x, validation_x, train_y, validation_y = sklearn.model_selection.train_test_split(x, y, test_size=validation_split, stratify=y)

In [ ]:
histories = [model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epochs, validation_data=(validation_x, validation_y), use_multiprocessing=use_multiprocessing, workers=os.cpu_count() - 1 if use_multiprocessing else 1) for model in models]

Epoch 1/20
800/800 [==============================] - 68s 84ms/step - loss: 0.1867 - accuracy: 0.9631 - auc: 0.5071 - val_loss: 0.1643 - val_accuracy: 0.9622 - val_auc: 0.5019
Epoch 2/20
800/800 [==============================] - 64s 80ms/step - loss: 0.1725 - accuracy: 0.9617 - auc: 0.5061 - val_loss: 0.1659 - val_accuracy: 0.9622 - val_auc: 0.5040
Epoch 3/20
800/800 [==============================] - 64s 80ms/step - loss: 0.1671 - accuracy: 0.9619 - auc: 0.5048 - val_loss: 0.1614 - val_accuracy: 0.9622 - val_auc: 0.5085
Epoch 4/20
800/800 [==============================] - 64s 79ms/step - loss: 0.1621 - accuracy: 0.9625 - auc: 0.5049 - val_loss: 0.1611 - val_accuracy: 0.9622 - val_auc: 0.5009
Epoch 5/20
800/800 [==============================] - 64s 80ms/step - loss: 0.1625 - accuracy: 0.9618 - auc: 0.5053 - val_loss: 0.1609 - val_accuracy: 0.9622 - val_auc: 0.5000
Epoch 6/20
800/800 [==============================] - 64s 80ms/step - loss: 0.1614 - accuracy: 0.9621 - auc: 0.4995 - va

## Test Model

In [ ]:
test_x, test_y = input_processing.get_data(A, 1000)

In [ ]:
histories = [model.evaluate(x=test_x, y=test_y, use_multiprocessing=use_multiprocessing, workers=os.cpu_count() - 1 if use_multiprocessing else 1) for model in models]

32/32 [==============================] - 1s 9ms/step - loss: 0.1157 - accuracy: 0.9760 - auc: 0.5000


In [ ]:
model = models[0]
model.save('/content/gdrive/My Drive/6.874 project/Colab/models/cnn')

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/6.874 project/Colab/models/cnn/assets
